In [2]:
import cv2
import numpy as np
from PIL import Image
from patchify import patchify
from tensorflow.keras.models import load_model

PATCH_SIZE = 480

def patch_image(image):

    instances = []

    if type(image) == str:
        image = cv2.imread(image)
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    size_x = (image.shape[1] // PATCH_SIZE) * PATCH_SIZE  # get width to nearest size divisible by patch size
    size_y = (image.shape[0] // PATCH_SIZE) * PATCH_SIZE  # get height to nearest size divisible by patch size

    image = Image.fromarray(image)

    # Crop original image to size divisible by patch size from top left corner
    image = np.array(image.crop((0, 0, size_x, size_y)))

    # Extract patches from each image, step=patch_size means no overlap
    patch_img = patchify(image, (PATCH_SIZE, PATCH_SIZE, 3), step=PATCH_SIZE)

    # iterate over vertical patch axis
    for j in range(patch_img.shape[0]):
        # iterate over horizontal patch axis
        for k in range(patch_img.shape[1]):
            # patches are located like a grid. use (j, k) indices to extract single patched image
            single_patch_img = patch_img[j, k]

            # Drop extra extra dimension from patchify
            instances.append(np.squeeze(single_patch_img))

    return instances, (int(size_x/PATCH_SIZE), int(size_y/PATCH_SIZE))

In [ ]:
def visualize(image):
    img = Image.fromarray(image)
    img.show()

In [ ]:
def depatchify(patches_arr, size, p_size=PATCH_SIZE):
    patches_np = np.array(patches_arr)

    w, h = size

    reshaped = patches_np.reshape(w, h, p_size, p_size, 3)

    bag = []

    for subindex in range(reshaped.shape[0]):
        line = np.concatenate(reshaped[subindex],axis=1)
        bag.append(line)

    image_to_reshape = np.array(bag)



    return image_to_reshape.reshape(w*p_size, h*p_size, 3)

In [ ]:
from enum import Enum
class MaskColorMap(Enum):
    Urban = (0, 255, 255) # Cyan
    Agriculture = (255, 255, 0) # Amarillo
    Rangeland = (255, 0, 255) # Morado
    Forest = (0, 255, 0) # Verde
    Water = (0, 0, 255) # Azul
    Barren = (255, 255, 255) # Blanco
    Uknown = (0,0,0) # Negro

In [ ]:
from tensorflow.keras.backend import flatten, sum
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger

def jaccard_index(y_true, y_pred):
    y_true_f = flatten(y_true)
    y_pred_f = flatten(y_pred)
    intersection = sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (sum(y_true_f) + sum(y_pred_f) - intersection + 1.0)

In [ ]:
def rgb_encode_mask(mask):
    # initialize rgb image with equal spatial resolution
    rgb_encode_image = np.zeros((mask.shape[0], mask.shape[1], 3))

    # iterate over MaskColorMap
    for j, cls in enumerate(MaskColorMap):
        # convert single integer channel to RGB channels
        rgb_encode_image[(mask == j)] = np.array(cls.value) / 255.
    return rgb_encode_image

In [ ]:
model = load_model('new_model_escrop_solo_0.h5', custom_objects={'jaccard_index': jaccard_index})

In [ ]:
patches, size = patch_image('dataset/11/train/28689_sat.jpg')
groundtruth_bgr = cv2.imread('dataset/11/masks/28689_mask.png')
groundtruth = cv2.cvtColor(groundtruth_bgr, cv2.COLOR_BGR2RGB)

In [ ]:
visualize(depatchify(patches, size))


In [ ]:
visualize(groundtruth)

In [ ]:
predictions = []
for patch in patches:
    pred = model.predict(np.expand_dims(patch, 0))
    pred = np.squeeze(pred)
    pred = np.argmax(pred, axis=-1)
    pred = rgb_encode_mask(pred)
    pred = pred*255
    predictions.append(pred)


In [ ]:
prediction = depatchify(predictions, size)

In [ ]:
p = np.array(predictions, dtype=np.uint8)

In [ ]:
image = depatchify(p, size)

In [ ]:
visualize(image)